In [114]:
from selenium import webdriver as web
from bs4 import BeautifulSoup
import time
import pandas
import json
import unidecode

In [140]:
# Acessar a página de cotações
option = web.ChromeOptions()
#option.headless = True
driver = web.Chrome(options=option)

In [141]:
driver.get("https://economia.awesomeapi.com.br/xml/available/uniq")
time.sleep(3)

main_element = driver.find_element_by_class_name("pretty-print")
html = main_element.get_attribute('outerHTML')
soup = BeautifulSoup(html, 'html.parser')




/tmp/ipykernel_6087/4146579865.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  main_element = driver.find_element_by_class_name("pretty-print")


In [142]:
class_line = soup.find_all('div', attrs={'class':"line"})
class_line

[<div class="line"><span class="folder-button fold"></span><span class="html-tag">&lt;xml&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;AFN&gt;</span><span>Afghani do Afeganistão</span><span class="html-tag">&lt;/AFN&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;MGA&gt;</span><span>Ariary Madagascarense</span><span class="html-tag">&lt;/MGA&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;THB&gt;</span><span>Baht Tailandês</span><span class="html-tag">&lt;/THB&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;PAB&gt;</span><span>Balboa Panamenho</span><span class="html-tag">&lt;/PAB&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;ETB&gt;</span><span>Birr Etíope</span><span class="html-tag">&lt;/ETB&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;BTC&gt;</span><span>Bitcoin</span><span class="html-tag">&lt;/BTC&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;BOB&gt;</span><sp

## Pegando os códigos de cada moeda

In [143]:
moedas = []
for line in class_line:
    moedas.append(line.find("span").get_text().strip("<>"))

In [144]:
moedas.pop(0)
moedas.pop(-1)

'/xml'

## Pegando os países das cotações da moeda

In [145]:
paises = []
for line in class_line:
    if len(line.find_all('span'))>1:
        paises.append(str(unidecode.unidecode(line.find_all("span")[1].get_text())))
    

In [146]:
paises.pop(0)

'<xml>'

In [147]:
driver.get("https://economia.awesomeapi.com.br/xml/available")
time.sleep(5)

main_element = driver.find_element_by_class_name("opened")
html = main_element.get_attribute('outerHTML')
soup2 = BeautifulSoup(html, 'html.parser')

driver.quit()

/tmp/ipykernel_6087/3313179147.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  main_element = driver.find_element_by_class_name("opened")


## Pegando as combinações de moedas

In [148]:
class_line = soup2.find_all('div', attrs={'class':"line"})
class_line

[<div class="line"><span class="html-tag">&lt;EUR-USD&gt;</span><span>Euro/Dólar Americano</span><span class="html-tag">&lt;/EUR-USD&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;DOGE-EUR&gt;</span><span>Dogecoin/Euro</span><span class="html-tag">&lt;/DOGE-EUR&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;USD-PLN&gt;</span><span>Dólar Americano/Zlóti Polonês</span><span class="html-tag">&lt;/USD-PLN&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;USD-BND&gt;</span><span>Dólar Americano/Dólar de Brunei</span><span class="html-tag">&lt;/USD-BND&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;USD-HRK&gt;</span><span>Dólar Americano/Kuna Croata</span><span class="html-tag">&lt;/USD-HRK&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;USD-SZL&gt;</span><span>Dólar Americano/Lilangeni Suazilandês</span><span class="html-tag">&lt;/USD-SZL&gt;</span></div>,
 <div class="line"><span class="html-tag">&lt;SAR-USD&g

In [149]:
combinacoes = []
for line in class_line:
    combinacoes.append(line.find("span").get_text().strip("<>"))


## Coletando as combinações por extenso

In [150]:
combinacoes_extenso = []
for line in class_line:
    if len(line.find_all('span'))>1:
        combinacoes_extenso.append(str(unidecode.unidecode(line.find_all("span")[1].get_text())))

Criando um dicionário com todos os valores encontrados

In [151]:
df_moedas = pandas.DataFrame(data = {"Cod_da_moeda": moedas, "Paises": paises})

In [152]:
df_combinacoes = pandas.DataFrame(data={"Combinacoes_de_conversao":combinacoes, "Combinacoes_por_extenso": combinacoes_extenso})

In [153]:
moedas_dict = {}
moedas_dict['Tipo da moeda'] = df_moedas.to_dict('records')

conversao_dict = {}
conversao_dict['Tipo de conversao'] = df_combinacoes.to_dict('records')

In [154]:
combinacoes_extenso

['Euro/Dolar Americano',
 'Dogecoin/Euro',
 'Dolar Americano/Zloti Polones',
 'Dolar Americano/Dolar de Brunei',
 'Dolar Americano/Kuna Croata',
 'Dolar Americano/Lilangeni Suazilandes',
 'Riyal Saudita/Dolar Americano',
 'Dolar Americano/Somoni do Tajiquistao',
 'Dolar Americano/Rublo Russo',
 'Hryvinia Ucraniana/Dolar Americano',
 'Dinar Iraquiano/Dolar Americano',
 'Real Brasileiro/Dolar Australiano',
 'Real Brasileiro/Dirham dos Emirados',
 'Real Brasileiro/Dinar do Bahrein',
 'Real Brasileiro/Riyal Saudita',
 'Euro/Coroa Norueguesa',
 'Euro/Florim Hungaro',
 'Euro/Yuan Chines',
 'Euro/Franco de Guine',
 'Euro/Rupia Indonesia',
 'Euro/Rial Iraniano',
 'Euro/Dolar Namibio',
 'Euro/Cordoba Nicaraguense',
 'Euro/Hryvinia Ucraniana',
 'Euro/Peso Uruguaio',
 'Euro/Manat Azeri',
 'Libra Esterlina/Real Brasileiro',
 'Franco Suico/Euro',
 'Dolar Americano/Rand Sul-Africano',
 'Dolar Americano/Coroa Sueca',
 'Dolar Americano/Dalasi da Gambia',
 'Dolar Americano/Dirham Marroquino',
 'Dolar A

In [132]:
js = json.dumps(moedas_dict)
fp = open('moedas.json', 'w')
fp.write(js)
fp.close()

In [133]:
js2 = json.dumps(conversao_dict)
fp = open('conversao.json', 'w')
fp.write(js2)
fp.close()